In [1]:
import time
from configparser import ConfigParser
from pfr_scraper import GameScraper, DatabaseMaker
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
from sqlalchemy import create_engine
from sqlalchemy.types import Float, Integer, String

In [2]:
page_off = 0
year1 = 1980
year2 = 2020
url = (
    'https://www.pro-football-reference.com/play-index/draft-finder.cgi?'
    f'request=1&year_min={year1}&year_max={year2}&pick_type=overall&pos%5B'
    '%5D=qb&pos%5B%5D=rb&pos%5B%5D=wr&pos%5B%5D=te&pos%5B%5D=e&pos%5B%5D=t'
    '&pos%5B%5D=g&pos%5B%5D=c&pos%5B%5D=ol&pos%5B%5D=dt&pos%5B%5D=de&pos'
    '%5B%5D=dl&pos%5B%5D=ilb&pos%5B%5D=olb&pos%5B%5D=lb&pos%5B%5D=cb&pos'
    '%5B%5D=s&pos%5B%5D=db&pos%5B%5D=k&pos%5B%5D=p&conference=any'
    f'&show=all&order_by=default&offset={page_off}'
)
col_list = [
    'player_id', 'player', 'year_id', 'draft_round', 'draft_pick', 'pos',
    'draft_age', 'college_id'
]

In [8]:
page = requests.get(url)
if page.status_code == 200:
    soup = BeautifulSoup(page.content, "lxml")
    table = soup.find('tbody')
    rows = table.findAll('tr')
df = pd.DataFrame(columns=col_list)

In [16]:
data_set = rows[0].findAll('td')
data_set[3]

<td class="left" csk="Burrow,Joe" data-append-csv="BurrJo01" data-stat="player"><a href="/players/B/BurrJo01.htm">Joe Burrow </a></td>

In [ ]:
for idx, item in enumerate(rows):
    data_set = rows[idx].findAll('td')
    if len(data_set) > 0:
        for idx, item in enumerate(data_set):
            if idx == 3:
                df.loc[count, 'player_id'] = item['data-append-csv']
                df.loc[count, 'player'] = item.text
            else:
                if item['data-stat'] in col:
                    try:
                        df.loc[count, item['data-stat']] = (
                            (datetime.strptime(item.text, '%Y-%m-%d') -
                                datetime(1970, 1, 1)).total_seconds()
                        )
                    except ValueError:
                        try:
                            df.loc[count, item['data-stat']] = (
                                round(float(item.text), 4)
                            )
                        except ValueError:
                            df.loc[count, item['data-stat']] = (
                                item.text
                            )
        count += 1

In [ ]:
    df = game_scrape.create_draft_df(col_list, int(counter))
    db_maker = DatabaseMaker(df)
    db_maker.create_draft_db()

In [ ]:
def create_draft_db(self):
    engine = create_engine('sqlite:///raw_draftlog.db', echo=True)
    table_name = 'draftlogs'
    self.df.to_sql(
        table_name,
        con=engine,
        if_exists='append',
        index=True,
        dtype={
            'player_id': String,
            'player': String,
            'year_id': Integer,
            'draft_round': Integer,
            'draft_pick': Integer,
            'pos': String,
            'draft_age': Integer,
            'college_id': String
        }
    )